# 注意力机制  
## Scaled Dot-Product Attention  
一个注意力函数，它的输入是Query、Key、Value，都是向量；经过注意力计算后，输出Output也是向量。  
![点积注意力](picture/scaled_dot_product_attention.jpg)  
Scales Dot-Product Attention(上图)，它包含以下组成部分：  
1、第一步，Query乘以Key，公式为$Q*K^T$。  
2、第二步，缩放，$Q*K^T$之后除以$\sqrt{d_k}$,其中d是Key的维度，即$\frac{Q*K^T}{\sqrt{d_k}}$。  
3、第三步（可选），乘以Attention的掩码Mask。  
4、第四步。取Softmax，把上面计算出来的值，变成0-1之间，且它们的和为1，$Softmax(\frac{Q*K^T}{\sqrt{d_k}})$。  
5、第五步，乘以Value，即$Softmax(\frac{Q*K^T}{\sqrt{d_k}})*V$。

In [1]:
import tensorflow as tf

In [2]:
#缩放点积注意力
def scaled_dot_product_attention(q, k, v, mask=None):
    """缩放点积注意力"""
    #第一步
    qk = tf.matmul(q, k, transpose_b=True)

    #第二步
    d_k = tf.cast(tf.shape(k)[-1], dtype=tf.float32)
    scaled_attention_logits = qk / tf.sqrt(d_k)

    #第三步
    if mask is not None:
        mask = tf.cast(mask, dtype=tf.float32)
        scaled_attention_logits += (mask*-1e9)

    #第四步
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    #第五步
    output = tf.matmul(attention_weights, v)
    return output, attention_weights

## Multi-Head Attention  
相比只与使用$d_model$维度的K、V和Q执行单个注意力函数不同，将Q、K，V分别用不同的维度，线性映射h次效果更好。  
![](picture/Multi_Head_Attention.jpg)  
多头注意力计算步骤：  
1、第一步，Q，K，V进行线性转化。  
2、第二步，Q，K，V进行多头转换。  
3、第三步，进行Attention计算。  
4、第四步，进行多头拼接。  
5、第五步，进行线性转换。

In [ ]:
class MultiHeadAttentiom(tf.keras.layers.Layer):
    """多头注意力"""
    def __init__(self, d_model, num_heads):
        self.d_model = d_model
        self.num_heads = num_heads
        assert self.d_model % self.num_heads == 0

        self.depth = self.d_model // self.num_heads

        #线性转换
        self.Wq = tf.keras.layers.Dense(self.d_model)
        self.Wk = tf.keras.layers.Dense(self.d_model)
        self.Wv = tf.keras.layers.Dense(self.d_model)

        self.dense = tf.keras.layers.Dense(self.d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q, k, v, mask=None):
        batch_size = tf.shape(q)[0]

        #第一步
        q = self.Wq(q)
        k = self.Wk(k)
        v = self.Wv(v)

        #第二步
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        #第三步
        scaled_attention_out, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        scaled_attention_out = tf.transpose(scaled_attention_out, perm=[0, 2, 1, 3])

        #第四步
        concat_attention_out = tf.reshape(scaled_attention_out, (batch_size, -1, self.d_model))

        #第五步
        output = self.dense(concat_attention_out)
        return output, attention_weights